<a href="https://colab.research.google.com/github/Prajwal011/LLM-s/blob/main/Rag_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q -U langchain-community \
              sentence-transformers\
              pymupdf\
              frontend\
              langchain_chroma \
              unstructured==0.7.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 25.7 MB/s eta 0:00:00


[How to import pdf documents in langchain](https://python.langchain.com/v0.2/docs/how_to/document_loader_pdf/#using-pypdf)



**Load whole Directory with multiple pdf's**

In [14]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("/content", glob="**/*.pdf")

docs = loader.load()

len(docs)

2

**Load single pdf**

In [15]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("expressions.pdf")

data = loader.load()

len(data)

1

**CharacterTextSplitter**

This is the most common and straightforward approach to chunking: we simply decide the number of tokens in our chunk and, optionally, whether there should be any overlap between them. In general, we will want to keep some overlap between chunks to make sure that the semantic context doesn’t get lost between chunks. Fixed-sized chunking will be the best path in most common cases. Compared to other forms of chunking, fixed-sized chunking is computationally cheap and simple to use since it doesn’t require the use of any NLP libraries

In [16]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=50)

text_chunks=text_splitter.split_documents(docs)

text_chunks[0]

Document(page_content='Expressions\nPython For Data Science\nWhat do you think when you hear the word expression. Most of the people start thinking about mathematical\nexpresssion. And you are not even wrong, you guessed it correct. So this chapter is about mathematical\nexpressions along with some basic programming concepts before you can do anything. Like any other beginner-\n\uf002\nSearch the docs ...\nin-training, you might think these concepts seem arcane and tedious, but with some knowledge and practice,\nyou’ll be able to command your computer to perform incredible feats.\nTOOLS\nThis chapter has a few examples that encourage you to type into the code cells, which lets you execute Python\nGoogle Colab\ninstructions one at a time and shows you the results instantly. Using the Jupyter Notebooks/Google Colab is great\nfor learning what basic Python instructions do, so give it a try as you follow along. You’ll remember the things you\nCHAPTERS\nDO much better than the things you on

**RecursiveCharacterTextSplitter**

Recursive chunking divides the input text into smaller chunks in a hierarchical and iterative manner using a set of separators. If the initial attempt at splitting the text doesn’t produce chunks of the desired size or structure, the method recursively calls itself on the resulting chunks with a different separator or criterion until the desired chunk size or structure is achieved. This means that while the chunks aren’t going to be exactly the same size, they’ll still “aspire” to be of a similar size.

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import Language
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)

texts = python_splitter.split_documents(docs)

texts[0]

Document(page_content='Expressions\n\nPython For Data Science\n\nWhat do you think when you hear the word expression. Most of the people start thinking about mathematical\n\nexpresssion. And you are not even wrong, you guessed it correct. So this chapter is about mathematical\n\nexpressions along with some basic programming concepts before you can do anything. Like any other beginner-\n\n\uf002\n\nSearch the docs ...\n\nin-training, you might think these concepts seem arcane and tedious, but with some knowledge and practice,\n\nyou’ll be able to command your computer to perform incredible feats.\n\nTOOLS\n\nThis chapter has a few examples that encourage you to type into the code cells, which lets you execute Python\n\nGoogle Colab\n\ninstructions one at a time and shows you the results instantly. Using the Jupyter Notebooks/Google Colab is great\n\nfor learning what basic Python instructions do, so give it a try as you follow along. You’ll remember the things you\n\nCHAPTERS\n\nDO much

Hopefully, this post will help you get a better intuition for how to approach chunking for your application.

[**Explore different types of chunking**](https://www.pinecone.io/learn/chunking-strategies/)



In [20]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_chroma import Chroma

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# db = Chroma.from_documents(texts, OpenAIEmbeddings(openai_api_key='Enter your open ai api key')) #### use if you have openai key

db = Chroma.from_documents(texts,embedding_function)

retriever = db.as_retriever(
    search_type="similarity",  # Also test "mmr"
    search_kwargs={"k": 8},
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import RetrievalQA
import os
import textwrap

repo_id = 'mistralai/Mistral-7B-Instruct-v0.2'

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ## hugginface api key

HUGGINGFACEHUB_API_TOKEN = ## hugginface api key

llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature=0.5, token=HUGGINGFACEHUB_API_TOKEN,
    # device=torch.device("gpu")
)

chain =  RetrievalQA.from_chain_type(llm=llm, chain_type = "stuff",return_source_documents=True, retriever=retriever)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [25]:
query = "give example of boolean operators"

result=chain({"query": query}, return_only_outputs=True)

wrapped_text = textwrap.fill(result['result'], width=500)

print(wrapped_text)

 The Boolean operators in Python are and, or, and not. The and operator evaluates to True if both operands are True. The or operator evaluates to True if at least one operand is True. The not operator evaluates to True if its operand is False. Here's an example of using these operators:  a = True b = False c = False  print(a and b) # False print(a and c) # False print(a or b) # True print(a or c) # True print(not a) # False print(not b) # True  You can also use these operators in more complex
expressions. For example:  x = 5 y = 10 z = 7  if (x > y) or (x < z): print("x is either greater than y or less than z")  In this example, the expression (x > y) or (x < z) is evaluated first, and since x is neither greater than y nor less than z, the whole expression is False. However, since the if statement allows for false conditions, the print statement is not executed.  You can also use nested if statements to create more complex conditions. For example:  age = 15  if age < 18: print("You
are